In [1]:
%load_ext autoreload
%autoreload 2

In [37]:
import checklist
import checklist.editor

In [38]:
a = checklist.editor.Editor()

In [43]:
a.lexicons['city']

['New York',
 'Los Angeles',
 'Chicago',
 'Houston',
 'Philadelphia',
 'Phoenix',
 'San Antonio',
 'San Diego',
 'Dallas',
 'San Jose',
 'Austin',
 'Indianapolis',
 'Jacksonville',
 'San Francisco',
 'Columbus',
 'Charlotte',
 'Fort Worth',
 'Detroit',
 'El Paso',
 'Memphis',
 'Seattle',
 'Denver',
 'Washington',
 'Boston',
 'Nashville-Davidson',
 'Baltimore',
 'Oklahoma City',
 'Louisville/Jefferson County',
 'Portland',
 'Las Vegas',
 'Milwaukee',
 'Albuquerque',
 'Tucson',
 'Fresno',
 'Sacramento',
 'Long Beach',
 'Kansas City',
 'Mesa',
 'Virginia Beach',
 'Atlanta',
 'Colorado Springs',
 'Omaha',
 'Raleigh',
 'Miami',
 'Oakland',
 'Minneapolis',
 'Tulsa',
 'Cleveland',
 'Wichita',
 'Arlington',
 'New Orleans',
 'Bakersfield',
 'Tampa',
 'Honolulu',
 'Aurora',
 'Anaheim',
 'Santa Ana',
 'St. Louis',
 'Riverside',
 'Corpus Christi',
 'Lexington-Fayette',
 'Pittsburgh',
 'Anchorage',
 'Stockton',
 'Cincinnati',
 'St. Paul',
 'Toledo',
 'Greensboro',
 'Newark',
 'Plano',
 'Henderson',

In [2]:
from nlp import checklist as cl

# Sentiment Analysis

In [44]:
suite_path = '/home/marcotcr/work/checklist/release_data/sentiment/sentiment_suite.pkl'

We have to tell `nlp` how to turn the data in a checklist into a map, which we do in the second argument.  
I'll just add this function to the checklist pickle file for my test suites, making the argument optional.

In [4]:
suite = cl.CheckListSuite(suite_path, lambda x: {'tweet': x })

Loading predictions from other models (which I have saved).  
I'm assuming that people would want to add predictions to `suite.dataset`, similar to the examples in [here](https://huggingface.co/nlp/processing.html#processing-data-row-by-row)

In [5]:
models = ['microsoft', 'google', 'amazon', 'bert', 'roberta']
for model in models:
    preds = open('/home/marcotcr/work/checklist/release_data/sentiment/predictions/%s' % model).read().splitlines()
    confs = [list(map(float, (x.split()[1:]))) for x in preds]
    preds = [int(x.split()[0]) for x in preds]
    conf_key = '%s_conf' % model
    suite.dataset = suite.dataset.map(lambda _, idx: {model: preds[idx], conf_key: confs[idx]}, with_indices=True)

After loading the predictions, we call `suite.compute` to compute test results for each model.  
The arguments are the prediction_key and the confidence_key.  
The second argument is optional, but many tests depend on having a confidence score to check for monotonicity, etc.

In [6]:
for model in models:
    conf_key = '%s_conf' % model
    suite.compute(model, conf_key)

There are more tests in this suite than I had in the paper, but let's pretend I want to replicate table 1 of the [checklist paper](https://homes.cs.washington.edu/~marcotcr/acl20_checklist.pdf).  
What I would do is look at the `suite.fail_rate` dict (I can change this name to `suite.results`, or whatever) for the different models.

In [7]:
checklist_table1 =  [
'neutral words in context',
'Sentiment-laden words in context',
'change neutral words with BERT',
'add positive phrases',
'add negative phrases',
'add random urls and handles',
'typos',
'change locations',
'change names',
'used to, but now',
'simple negations: not negative',
'simple negations: not neutral is still neutral',
'simple negations: I thought x was negative, but it was not (should be neutral or positive)',
'Hard: Negation of positive with neutral stuff in the middle (should be negative)',
'my opinion is what matters',
'Q & A: yes',
'Q & A: no',
]

In [8]:
print (' '.join([x[:5] for x in models]))
for t in checklist_table1:
    r = ' '.join(['%5.1f' % (suite.fail_rate[m][t]) for m in models])
    print('%s %s' % (r, t))

micro googl amazo bert rober
  0.0   7.6   4.8  94.6  81.8 neutral words in context
  4.0  15.0   2.8   0.0   0.2 Sentiment-laden words in context
  9.4  16.2  12.4  10.2  10.2 change neutral words with BERT
 12.6  12.4   1.4   0.2  10.2 add positive phrases
  0.8  34.6   5.0   0.0  13.2 add negative phrases
  9.6  13.4  24.8  11.4   7.4 add random urls and handles
  5.6  10.2  10.4   5.2   3.8 typos
  7.0  20.8  14.8   7.6   6.4 change locations
  2.4  15.1   9.1   6.6   2.4 change names
 41.0  36.6  42.2  18.8  11.0 used to, but now
 18.8  54.2  29.4  13.2   2.6 simple negations: not negative
 40.4  39.6  74.2  98.4  95.4 simple negations: not neutral is still neutral
100.0  90.4 100.0  84.8   7.2 simple negations: I thought x was negative, but it was not (should be neutral or positive)
 98.4 100.0 100.0  74.0  30.2 Hard: Negation of positive with neutral stuff in the middle (should be negative)
 45.4  62.4  68.0  38.8  30.0 my opinion is what matters
  9.0  57.6  20.8   3.6   3.0 Q 

Let's suppose I want to compare the pipeline in transformers to these models.  
This test suite assumes the labels are [negative, neutral, positive], so we have to do some converting:

In [9]:
from transformers import pipeline
model = pipeline("sentiment-analysis", device=0)

In [10]:
import numpy as np
def pred_and_conf(data):
    # change format to softmax, make everything in [0.33, 0.66] range be predicted as neutral
    preds = model(data)
    pr = np.array([x['score'] if x['label'] == 'POSITIVE' else 1 - x['score'] for x in preds])
    pp = np.zeros((pr.shape[0], 3))
    margin_neutral = 1/3.
    mn = margin_neutral / 2.
    neg = pr < 0.5 - mn
    pp[neg, 0] = 1 - pr[neg]
    pp[neg, 2] = pr[neg]
    pos = pr > 0.5 + mn
    pp[pos, 0] = 1 - pr[pos]
    pp[pos, 2] = pr[pos]
    neutral_pos = (pr >= 0.5) * (pr < 0.5 + mn)
    pp[neutral_pos, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_pos] - 0.5)
    pp[neutral_pos, 2] = 1 - pp[neutral_pos, 1]
    neutral_neg = (pr < 0.5) * (pr > 0.5 - mn)
    pp[neutral_neg, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_neg] - 0.5)
    pp[neutral_neg, 0] = 1 - pp[neutral_neg, 1]
    preds = np.argmax(pp, axis=1)
    return preds, pp

Add predictions to `suite.dataset`

In [11]:
def add_pipeline(x):
    preds, confs = pred_and_conf(x['tweet'])
    return {'hf_pipeline': preds, 'hf_pipeline_conf': confs}
suite.dataset = suite.dataset.map(add_pipeline , batched=True)

In [12]:
suite.compute('hf_pipeline', 'hf_pipeline_conf')

In [13]:
models.append('hf_pipeline')

In [14]:
print (' '.join([x[:5] for x in models]))
for t in checklist_table1:
    r = ' '.join(['%5.1f' % (suite.fail_rate[m][t]) for m in models])
    print('%s %s' % (r, t))

micro googl amazo bert rober hf_pi
  0.0   7.6   4.8  94.6  81.8  95.8 neutral words in context
  4.0  15.0   2.8   0.0   0.2   0.8 Sentiment-laden words in context
  9.4  16.2  12.4  10.2  10.2   9.8 change neutral words with BERT
 12.6  12.4   1.4   0.2  10.2   0.0 add positive phrases
  0.8  34.6   5.0   0.0  13.2   6.8 add negative phrases
  9.6  13.4  24.8  11.4   7.4  15.4 add random urls and handles
  5.6  10.2  10.4   5.2   3.8   6.6 typos
  7.0  20.8  14.8   7.6   6.4  10.0 change locations
  2.4  15.1   9.1   6.6   2.4   5.1 change names
 41.0  36.6  42.2  18.8  11.0  32.6 used to, but now
 18.8  54.2  29.4  13.2   2.6  12.8 simple negations: not negative
 40.4  39.6  74.2  98.4  95.4  97.4 simple negations: not neutral is still neutral
100.0  90.4 100.0  84.8   7.2 100.0 simple negations: I thought x was negative, but it was not (should be neutral or positive)
 98.4 100.0 100.0  74.0  30.2  86.8 Hard: Negation of positive with neutral stuff in the middle (should be negative)

### Using marcotcr/checklist

Users can access my package's object if they want to use it (e.g. for visualizations), with the caveat that it doesn't really allow for model comparison (it only keeps the state of the last model we called `compute` on).  
For example:

In [15]:
type(suite.suite)

checklist.test_suite.TestSuite

In [16]:
suite.suite.tests['Sentiment-laden words in context'].summary()

Test cases:      8658
Test cases run:  500
Fails (rate):    4 (0.8%)

Example fails:
0.0 0.0 1.0 The flight is average.
----
0.1 0.0 0.9 This was an average staff.
----
0.1 0.0 0.9 This food was average.
----


In [17]:
suite.summary() # calls suite.suite.summary

Vocabulary

single positive words
Test cases:      34
Fails (rate):    0 (0.0%)


single negative words
Test cases:      35
Fails (rate):    1 (2.9%)

Example fails:
0.3 0.0 0.7 average
----


single neutral words
Test cases:      13
Fails (rate):    13 (100.0%)

Example fails:
0.0 0.0 1.0 found
----
0.0 0.0 1.0 international
----
0.0 0.0 1.0 Italian
----


Sentiment-laden words in context
Test cases:      8658
Test cases run:  500
Fails (rate):    4 (0.8%)

Example fails:
0.0 0.0 1.0 That was a weird food.
----
0.1 0.0 0.9 This food was average.
----
0.0 0.0 1.0 The flight is average.
----


neutral words in context
Test cases:      1716
Test cases run:  500
Fails (rate):    479 (95.8%)

Example fails:
0.0 0.0 1.0 We found this airline.
----
1.0 0.0 0.0 That was a commercial pilot.
----
0.0 0.0 1.0 We saw that flight.
----


intensifiers
Test cases:      2000
Test cases run:  500
After filtering: 496 (99.2%)
Fails (rate):    8 (1.6%)

Example fails:
0.9 0.0 0.1 That was an average com

### Slicing tests on nlp.checklist

Examples of a certain test

In [18]:
simple = suite.get_test('Sentiment-laden words in context')

In [19]:
simple['tweet'][:10]

['I valued the flight.',
 'That is a sad customer service.',
 'We like the flight.',
 'This was a nice crew.',
 'We abhor that flight.',
 'This staff is difficult.',
 'I valued that aircraft.',
 'This was a fantastic flight.',
 'I hate the food.',
 'I despised the food.']

Filtering by examples where google fails and hf does not:

In [20]:
google_fails_hf_doesnt = simple.filter(lambda x:x['fail']['hf_pipeline'] == 0 and x['fail']['google'] == 1)

In [21]:
mapz = ['negative', 'neutral', 'positive']
for x in np.random.choice(google_fails_hf_doesnt.shape[0], 5):
    x = google_fails_hf_doesnt[int(x)]
    print('HF:%-8s GOOGLE:%-8s %s' % (mapz[x['hf_pipeline']], mapz[x['google']], x['tweet']))

HF:negative GOOGLE:neutral  The crew is average.
HF:negative GOOGLE:neutral  It is a hard aircraft.
HF:negative GOOGLE:neutral  That is a creepy plane.
HF:negative GOOGLE:neutral  That is an ugly cabin crew.
HF:negative GOOGLE:neutral  This airline is hard.


Perturbation tests combine multiple examples, so looking at a single row in the dataset would not give us a good picture.  
Instead, we want to aggregate each testcase into a row of examples (data goes into the `data` key):

In [22]:
perturbation = suite.get_test('change locations', aggregate_testcases=True)

In [23]:
any_fails = lambda x, key: any([y['fail'][key] for y in x['data']])
google_fails_hf_doesnt = perturbation.filter(lambda x: not any_fails(x, 'hf_pipeline') and any_fails(x, 'google'))

This is an INV test, so a failure is a change in prediction. Some examples:

In [25]:
mapz = ['negative', 'neutral', 'positive']
for x in np.random.choice(google_fails_hf_doesnt.shape[0], 5):
    x = google_fails_hf_doesnt[int(x)]
    orig = x['data'][0]
    fail = [y for y in x['data'] if y['fail']['google']][0]
    print('HF:%-8s GOOGLE:%-8s %s' % (mapz[orig['hf_pipeline']], mapz[orig['google']], orig['tweet']))
    print('HF:%-8s GOOGLE:%-8s %s' % (mapz[fail['hf_pipeline']], mapz[fail['google']], fail['tweet']))
    print()

HF:negative GOOGLE:positive @SouthwestAir I can easily get to the Atlanta show, I just need tickets, help?!
HF:negative GOOGLE:neutral  @SouthwestAir I can easily get to the Jefferson City show, I just need tickets, help?!

HF:positive GOOGLE:neutral  @SouthwestAir how can customers get in touch with you internationally from Mexico for lost baggage
HF:positive GOOGLE:positive @SouthwestAir how can customers get in touch with you internationally from Colombia for lost baggage

HF:positive GOOGLE:neutral  @SouthwestAir Denver to Phoenix, I think we're finally getting ready to take off.
HF:positive GOOGLE:positive @SouthwestAir Denver to Buena Park, I think we're finally getting ready to take off.

HF:negative GOOGLE:negative .@united Nope - had to rebook through Houston so I could get to Amarillo before 11pm. Stuck in SFO for four hours.
HF:negative GOOGLE:neutral  .@united Nope - had to rebook through Bell Gardens so I could get to Amarillo before 11pm. Stuck in SFO for four hours.

HF:

# SQuAD

In [26]:
suite_path = '/home/marcotcr/work/checklist/release_data/squad/squad_suite.pkl'

Examples in this suite are tuples of (context, question)

In [27]:
suite = cl.CheckListSuite(suite_path, lambda x: {'context': x[0], 'question': x[1]})

In [28]:
bert_preds = open('/home/marcotcr/work/checklist/release_data/squad/predictions/bert').read().splitlines()

In [29]:
suite.dataset = suite.dataset.map(lambda _, idx: {'bert': bert_preds[idx]}, with_indices=True)

These tests don't really require confidence scores

In [30]:
suite.compute('bert')

In [31]:
checklist_table3 = [
    'A is COMP than B. Who is more / less COMP?',
    'Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?',
    'size, shape, age, color',
    'Profession vs nationality',
    'Animal vs Vehicle v2',
    'A is COMP than B. Who is antonym(COMP)? B',
    'A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.',
    'Question typo',
    'Add random sentence to context',
    'There was a change in profession',
    'Understanding before / after -> first / last.',
    'Negation in context, may or may not be in question',
    'Negation in question only.', 'M/F failure rates should be similar for different professions',
    'Basic coref, he / she',
    'Basic coref, his / her',
    'Former / Latter',
    'Agent / object distinction',
    'Agent / object distinction with 3 agents'
]

In [34]:
for t in checklist_table3:
    print('%5.1f %s' % (suite.fail_rate['bert'][t], t))

 20.0 A is COMP than B. Who is more / less COMP?
 91.3 Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?
 82.4 size, shape, age, color
 49.4 Profession vs nationality
 26.2 Animal vs Vehicle v2
 67.3 A is COMP than B. Who is antonym(COMP)? B
100.0 A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.
 11.6 Question typo
  9.8 Add random sentence to context
 41.5 There was a change in profession
 82.9 Understanding before / after -> first / last.
 67.5 Negation in context, may or may not be in question
100.0 Negation in question only.
 46.2 M/F failure rates should be similar for different professions
100.0 Basic coref, he / she
 91.8 Basic coref, his / her
100.0 Former / Latter
 60.8 Agent / object distinction
 95.7 Agent / object distinction with 3 agents


# TODO

- Put some data into `suite.dataset.info`, so people know what the predictions and confidences should look like (e.g. [0, 1, 2] for sentiment, or string for SQuAD)
- Warn people that CheckList suites contained pickled functions, which may not be safe
- Write some documentation for `nlp.checklist`
- Clean up code for `nlp.checklist`, include typing, etc
- Write up a better example notebook.
- Figure out how to host checklists
- Improve checklist browsing (small change over nlp.dataset browsing)